In [1]:
# use BioSent2Vec and traditional ml on titles
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pickle

In [2]:
import sent2vec

In [3]:
neg_path = '/home/paperspace/Documents/RCNN-421-BioSentVec/data/title.neg-ns'
pos_path = '/home/paperspace/Documents/RCNN-421-BioSentVec/data/title.pos-ns'
model_path = 'BioSentVec_PubMed_MIMICIII-bigram_d700.bin'
model = sent2vec.Sent2vecModel()

In [4]:
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [5]:
# prepare the data from one single file
# 60/20/20
pos_arrays = np.zeros((23499, 700)) # the sum of all the training examples (neg+pos)
pos_labels = np.zeros(23499)

neg_arrays = np.zeros((23499, 700)) # the sum of all the training examples (neg+pos)
neg_labels = np.zeros(23499)

In [6]:
with open(pos_path) as pos:
    i = 0
    for line in pos:
        #print(line)
        sentence_vector = model.embed_sentence(line)
        pos_arrays[i] = sentence_vector
        pos_labels[i] = 1
        i = i + 1
    print(i)

23499


In [7]:
with open(neg_path) as neg:
    i = 0
    for line in neg:
    	sentence_vector = model.embed_sentence(line)
    	neg_arrays[i] = sentence_vector
    	neg_labels[i] = 0
    	i = i + 1
    print(i)

23499


In [8]:
np.random.seed(10)
dev_sample_percentage = 0.2
test_sample_percentage = 0.2
total = 23499 
np.random.shuffle(pos_arrays)
np.random.shuffle(neg_arrays)
dev_sample_index = -1 * int(dev_sample_percentage * float(total))
test_sample_index = -1 * int(test_sample_percentage * float(total)) 

In [9]:
training_pos, val_pos, test_pos  = pos_arrays[:dev_sample_index+test_sample_index], pos_arrays[dev_sample_index+test_sample_index:test_sample_index], pos_arrays[test_sample_index:] 
training_neg, val_neg, test_neg  = neg_arrays[:dev_sample_index+test_sample_index], neg_arrays[dev_sample_index+test_sample_index:test_sample_index], neg_arrays[test_sample_index:] 


In [10]:
# https://stackoverflow.com/questions/3674409/how-to-split-partition-a-dataset-into-training-and-test-datasets-for-e-g-cros
train_arrays = np.concatenate([training_pos, training_neg])
print(np.shape(train_arrays))
train_label_num, feature_dim = np.shape(train_arrays)
print(train_label_num)

(28202, 700)
28202


In [11]:
val_arrays = np.concatenate([val_pos, val_neg])
print(np.shape(val_arrays))
val_label_num, feature_dim = np.shape(val_arrays)
print(val_label_num)

(9398, 700)
9398


In [12]:
test_arrays = np.concatenate([test_pos, test_neg])
print(np.shape(test_arrays))
test_label_num, feature_dim = np.shape(test_arrays)
print(test_label_num)

(9398, 700)
9398


In [13]:
train_pos_label = np.ones(int(train_label_num/2))
train_neg_label = np.zeros(int(train_label_num/2))
train_labels = np.concatenate([train_pos_label, train_neg_label])

val_pos_label = np.ones(int(val_label_num/2))
val_neg_label = np.zeros(int(val_label_num/2))
val_labels = np.concatenate([val_pos_label, val_neg_label])

test_pos_label = np.ones(int(test_label_num/2))
test_neg_label = np.zeros(int(test_label_num/2))
test_labels = np.concatenate([test_pos_label, test_neg_label])

In [14]:
print(len(train_arrays))
print(len(val_arrays))
print(len(test_arrays))

28202
9398
9398


In [15]:
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score
import pickle
import glob, os

In [ ]:
# ensemble learning
from sklearn import svm
abs_dir = "/home/paperspace/Documents/BioSentVec/"
lr_dir = abs_dir + "lg_models/"
rf_dir = abs_dir + "rf_models/"
svm_dir = abs_dir + "svm_models_linear/"# svm_models_linear/c_0.1_svm.sav

lr_model_1 = pickle.load(open(lr_dir + "0.01_lr.sav", 'rb'))
#rf_model = pickle.load(open(rf_dir + "d_6_200_rf.sav", 'rb'))
svm_model_1 = pickle.load(open(svm_dir + "c_0.1_svm.sav", 'rb'))


In [27]:
# logistic regression
lr_save_dir = "/home/paperspace/Documents/BioSentVec/lg_models/"
C_values = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
for each in C_values:
    # logistic regression
    print("*****************************************************")
    print("current C = " + str(each))
    logistic_model = LogisticRegression(C=each, random_state=0, class_weight='balanced')
    # Train model
    model = logistic_model.fit(train_arrays, train_labels)
    Y_train_pred = logistic_model.predict(train_arrays)
    acc_train, pre_train, rec_train, f1_train = accuracy_score(train_labels, Y_train_pred), precision_score(train_labels, Y_train_pred), recall_score(train_labels, Y_train_pred), f1_score(train_labels, Y_train_pred)
    print('Train  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_train, pre_train, rec_train, f1_train))
    Y_val_pred = logistic_model.predict(val_arrays)
    acc_val, pre_val, rec_val, f1_val = accuracy_score(val_labels, Y_val_pred), precision_score(val_labels, Y_val_pred), recall_score(val_labels, Y_val_pred), f1_score(val_labels, Y_val_pred)
    print('Validation  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_val, pre_val, rec_val, f1_val))
    Y_test_pred = logistic_model.predict(test_arrays)
    acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred), recall_score(test_labels, Y_test_pred), f1_score(test_labels, Y_test_pred)
    print('Test  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_test, pre_test, rec_test, f1_test))
    filename = str(each) + '_lr.sav'
    pickle.dump(model, open(lr_save_dir + filename, 'wb'))
    print("current model saved to " + lr_save_dir + filename)
print("------------------------Work done-------------------------------------")

*****************************************************
current C = 1e-05


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train  | Acc: 0.67505851 | Precision: 0.69490722 | Recall: 0.62414013 | F1: 0.65762535
Validation  | Acc: 0.67152586 | Precision: 0.69000471 | Recall: 0.62289849 | F1: 0.65473661
Test  | Acc: 0.67695254 | Precision: 0.69792906 | Recall: 0.62396255 | F1: 0.65887640
current model saved to /home/paperspace/Documents/BioSentVec/lg_models/1e-05_lr.sav
*****************************************************
current C = 0.0001


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train  | Acc: 0.67824977 | Precision: 0.67944599 | Recall: 0.67491667 | F1: 0.67717376
Validation  | Acc: 0.67716535 | Precision: 0.67611593 | Recall: 0.68014471 | F1: 0.67812434
Test  | Acc: 0.68046393 | Precision: 0.68111918 | Recall: 0.67865503 | F1: 0.67988487
current model saved to /home/paperspace/Documents/BioSentVec/lg_models/0.0001_lr.sav
*****************************************************
current C = 0.001


...

Train  | Acc: 0.72150911 | Precision: 0.72217085 | Recall: 0.72001986 | F1: 0.72109375
Validation  | Acc: 0.70068100 | Precision: 0.69928149 | Recall: 0.70419238 | F1: 0.70172834
Test  | Acc: 0.70600128 | Precision: 0.70844100 | Recall: 0.70014897 | F1: 0.70427058
current model saved to /home/paperspace/Documents/BioSentVec/lg_models/10_lr.sav
*****************************************************
current C = 100


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train  | Acc: 0.72161549 | Precision: 0.72223012 | Recall: 0.72023261 | F1: 0.72122998
Validation  | Acc: 0.70057459 | Precision: 0.69913374 | Recall: 0.70419238 | F1: 0.70165394
Test  | Acc: 0.70589487 | Precision: 0.70828848 | Recall: 0.70014897 | F1: 0.70419521
current model saved to /home/paperspace/Documents/BioSentVec/lg_models/100_lr.sav
*****************************************************
current C = 1000


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train  | Acc: 0.72161549 | Precision: 0.72223012 | Recall: 0.72023261 | F1: 0.72122998
Validation  | Acc: 0.70046818 | Precision: 0.69907016 | Recall: 0.70397957 | F1: 0.70151628
Test  | Acc: 0.70589487 | Precision: 0.70828848 | Recall: 0.70014897 | F1: 0.70419521
current model saved to /home/paperspace/Documents/BioSentVec/lg_models/1000_lr.sav
------------------------Work done-------------------------------------


In [32]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
depth =list(range(4,9,1)) #list(range(5,40,5))
num_of_trees = list(range(50, 250, 50)) # list(range(20, 300, 20))
rf_save_dir = "/home/paperspace/Documents/BioSentVec/rf_models/"
for d in depth:
    for n in num_of_trees:
        print("************************************************************************")
        print("depth = " + str(d))
        print("num of trees = " + str(n))
        forest_model = RandomForestClassifier(n_estimators=n, max_depth = d, random_state=0,class_weight="balanced", n_jobs=-1) #using all processors
        model = forest_model.fit(train_arrays, train_labels)
        Y_train_pred = forest_model.predict(train_arrays)
        acc_train, pre_train, rec_train, f1_train = accuracy_score(train_labels, Y_train_pred), precision_score(train_labels, Y_train_pred), recall_score(train_labels, Y_train_pred), f1_score(train_labels, Y_train_pred)
        print('Train  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_train, pre_train, rec_train, f1_train))
        Y_val_pred = forest_model.predict(val_arrays)
        acc_val, pre_val, rec_val, f1_val = accuracy_score(val_labels, Y_val_pred), precision_score(val_labels, Y_val_pred), recall_score(val_labels, Y_val_pred), f1_score(val_labels, Y_val_pred)
        print('Validation  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_val, pre_val, rec_val, f1_val))
        Y_test_pred = forest_model.predict(test_arrays)
        acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred), recall_score(test_labels, Y_test_pred), f1_score(test_labels, Y_test_pred)
        print('Test  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_test, pre_test, rec_test, f1_test))
        filename = "d"+"_" + str(d) + "_" + str(n) + '_rf.sav'
        pickle.dump(model, open(rf_save_dir + filename, 'wb'))
        print("current model saved to " + rf_save_dir + filename)

print("-----------------------------------Work done--------------------------------------")

************************************************************************
depth = 4
num of trees = 50
Train  | Acc: 0.69154670 | Precision: 0.69637924 | Recall: 0.67924261 | F1: 0.68770418
Validation  | Acc: 0.66716323 | Precision: 0.66903379 | Recall: 0.66163013 | F1: 0.66531136
Test  | Acc: 0.66961056 | Precision: 0.67439825 | Recall: 0.65588423 | F1: 0.66501241
current model saved to /home/paperspace/Documents/BioSentVec/rf_models/d_4_50_rf.sav
************************************************************************
depth = 4
num of trees = 100
Train  | Acc: 0.69381604 | Precision: 0.69879255 | Recall: 0.68129920 | F1: 0.68993501
Validation  | Acc: 0.66790807 | Precision: 0.67026327 | Recall: 0.66099170 | F1: 0.66559520
Test  | Acc: 0.67407959 | Precision: 0.67852466 | Recall: 0.66163013 | F1: 0.66997091
current model saved to /home/paperspace/Documents/BioSentVec/rf_models/d_4_100_rf.sav
************************************************************************
depth = 4
num of trees 

Train  | Acc: 0.80653854 | Precision: 0.81997187 | Recall: 0.78554712 | F1: 0.80239044
Validation  | Acc: 0.68791232 | Precision: 0.68779243 | Recall: 0.68823154 | F1: 0.68801191
Test  | Acc: 0.69067887 | Precision: 0.69145299 | Recall: 0.68865716 | F1: 0.69005224
current model saved to /home/paperspace/Documents/BioSentVec/rf_models/d_8_150_rf.sav
************************************************************************
depth = 8
num of trees = 200
Train  | Acc: 0.80735409 | Precision: 0.82141798 | Recall: 0.78547621 | F1: 0.80304513
Validation  | Acc: 0.69025325 | Precision: 0.68980892 | Recall: 0.69142371 | F1: 0.69061537
Test  | Acc: 0.69227495 | Precision: 0.69284952 | Recall: 0.69078527 | F1: 0.69181586
current model saved to /home/paperspace/Documents/BioSentVec/rf_models/d_8_200_rf.sav
-----------------------------------Work done--------------------------------------


In [16]:
# SVM
from sklearn import svm
C_values = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
svm_save_dir = "/home/paperspace/Documents/BioSentVec/svm_models_linear/"
for c in C_values:
        print("************************************************************************")
        print("c = " + str(c))
        svm_model = svm.LinearSVC(random_state=0, C=c)
        model = svm_model.fit(train_arrays, train_labels)
        Y_train_pred = svm_model.predict(train_arrays)
        acc_train, pre_train, rec_train, f1_train = accuracy_score(train_labels, Y_train_pred), precision_score(train_labels, Y_train_pred), recall_score(train_labels, Y_train_pred), f1_score(train_labels, Y_train_pred)
        print('Train  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_train, pre_train, rec_train, f1_train))
        Y_val_pred = svm_model.predict(val_arrays)
        acc_val, pre_val, rec_val, f1_val = accuracy_score(val_labels, Y_val_pred), precision_score(val_labels, Y_val_pred), recall_score(val_labels, Y_val_pred), f1_score(val_labels, Y_val_pred)
        print('Validation  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_val, pre_val, rec_val, f1_val))
        Y_test_pred = svm_model.predict(test_arrays)
        acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred), recall_score(test_labels, Y_test_pred), f1_score(test_labels, Y_test_pred)
        print('Test  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_test, pre_test, rec_test, f1_test))
        filename = "c"+"_" + str(c) + '_svm.sav'
        pickle.dump(model, open(svm_save_dir + filename, 'wb'))
        print("current model saved to " + svm_save_dir + filename)

print("-----------------------------------Work done--------------------------------------")

************************************************************************
c = 1e-05
Train  | Acc: 0.67746968 | Precision: 0.67966114 | Recall: 0.67137082 | F1: 0.67549055
Validation  | Acc: 0.67642052 | Precision: 0.67653322 | Recall: 0.67610130 | F1: 0.67631719
Test  | Acc: 0.68014471 | Precision: 0.68192564 | Recall: 0.67525005 | F1: 0.67857143
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_1e-05_svm.sav
************************************************************************
c = 0.0001
Train  | Acc: 0.69317779 | Precision: 0.69218287 | Recall: 0.69576626 | F1: 0.69396994
Validation  | Acc: 0.69216855 | Precision: 0.68836045 | Recall: 0.70227708 | F1: 0.69524913
Test  | Acc: 0.69238136 | Precision: 0.69031579 | Recall: 0.69780804 | F1: 0.69404170
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_0.0001_svm.sav
************************************************************************
c = 0.001
Train  | Acc: 0.71257358 | Pr

/home/paperspace/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train  | Acc: 0.72182824 | Precision: 0.72282376 | Recall: 0.71959436 | F1: 0.72120544
Validation  | Acc: 0.70121302 | Precision: 0.70010582 | Recall: 0.70397957 | F1: 0.70203735
Test  | Acc: 0.70589487 | Precision: 0.70864783 | Recall: 0.69929772 | F1: 0.70394173
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_0.1_svm.sav
************************************************************************
c = 1


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train  | Acc: 0.72154457 | Precision: 0.72225384 | Recall: 0.71994894 | F1: 0.72109955
Validation  | Acc: 0.70142584 | Precision: 0.70040229 | Recall: 0.70397957 | F1: 0.70218637
Test  | Acc: 0.70515003 | Precision: 0.70784821 | Recall: 0.69865929 | F1: 0.70322373
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_1_svm.sav
************************************************************************
c = 10


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train  | Acc: 0.71048153 | Precision: 0.74216710 | Recall: 0.64506063 | F1: 0.69021512
Validation  | Acc: 0.69578634 | Precision: 0.72190063 | Recall: 0.63694403 | F1: 0.67676653
Test  | Acc: 0.69876569 | Precision: 0.72747199 | Recall: 0.63566716 | F1: 0.67847814
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_10_svm.sav
************************************************************************
c = 100


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train  | Acc: 0.62825332 | Precision: 0.66460362 | Recall: 0.51783561 | F1: 0.58211097
Validation  | Acc: 0.62523941 | Precision: 0.65790180 | Recall: 0.52181315 | F1: 0.58200807
Test  | Acc: 0.61342839 | Precision: 0.64570804 | Recall: 0.50266014 | F1: 0.56527462
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_100_svm.sav
************************************************************************
c = 1000
Train  | Acc: 0.62970711 | Precision: 0.60879134 | Recall: 0.72583505 | F1: 0.66218096
Validation  | Acc: 0.61896148 | Precision: 0.59862385 | Recall: 0.72206853 | F1: 0.65457702
Test  | Acc: 0.62140881 | Precision: 0.60056408 | Recall: 0.72504788 | F1: 0.65696105
current model saved to /home/paperspace/Documents/BioSentVec/svm_models_linear/c_1000_svm.sav
-----------------------------------Work done--------------------------------------


/home/paperspace/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SVM
from sklearn import svm
C_2d_range = [2,4,8,16] #[0.0005, 0.01, 0.05, 0.1, 0.2, 0.5, 1,2,4,8,16]
gamma_2d_range = [0.0001, 0.001, 0.01, 0.1, 1,10, 100, 1000]
svm_save_dir = "/home/paperspace/Documents/BioSentVec/svm_models/"
for c in C_2d_range:
    for g in gamma_2d_range:
        print("************************************************************************")
        print("c = " + str(c))
        print("gamma = " + str(g))
        svm_model = svm.SVC(C=c, gamma=g)
        model = svm_model.fit(train_arrays, train_labels)
        Y_train_pred = svm_model.predict(train_arrays)
        acc_train, pre_train, rec_train, f1_train = accuracy_score(train_labels, Y_train_pred), precision_score(train_labels, Y_train_pred), recall_score(train_labels, Y_train_pred), f1_score(train_labels, Y_train_pred)
        print('Train  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_train, pre_train, rec_train, f1_train))
        Y_val_pred = svm_model.predict(val_arrays)
        acc_val, pre_val, rec_val, f1_val = accuracy_score(val_labels, Y_val_pred), precision_score(val_labels, Y_val_pred), recall_score(val_labels, Y_val_pred), f1_score(val_labels, Y_val_pred)
        print('Validation  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_val, pre_val, rec_val, f1_val))
        Y_test_pred = svm_model.predict(test_arrays)
        acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred), recall_score(test_labels, Y_test_pred), f1_score(test_labels, Y_test_pred)
        print('Test  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_test, pre_test, rec_test, f1_test))
        filename = "c"+"_" + str(c) + "_g_" + str(g) + '_svm.sav'
        pickle.dump(model, open(svm_save_dir + filename, 'wb'))
        print("current model saved to " + svm_save_dir + filename)

print("-----------------------------------Work done--------------------------------------")

************************************************************************
c = 0.0005
gamma = 0.0001
Train  | Acc: 0.65435076 | Precision: 0.62402416 | Recall: 0.77661159 | F1: 0.69200632
Validation  | Acc: 0.65141519 | Precision: 0.62152007 | Recall: 0.77442009 | F1: 0.68959636
Test  | Acc: 0.65428815 | Precision: 0.62388927 | Recall: 0.77697382 | F1: 0.69206710
current model saved to /home/paperspace/Documents/BioSentVec/svm_models/c_0.0005_g_0.0001_svm.sav
************************************************************************
c = 0.0005
gamma = 0.001
Train  | Acc: 0.66697397 | Precision: 0.64733121 | Recall: 0.73363591 | F1: 0.68778672
Validation  | Acc: 0.66429027 | Precision: 0.64473191 | Recall: 0.73185784 | F1: 0.68553773
Test  | Acc: 0.66577995 | Precision: 0.64626361 | Recall: 0.73249628 | F1: 0.68668329
current model saved to /home/paperspace/Documents/BioSentVec/svm_models/c_0.0005_g_0.001_svm.sav
************************************************************************
c = 0

Train  | Acc: 0.69604993 | Precision: 0.69519170 | Recall: 0.69824835 | F1: 0.69671667
Validation  | Acc: 0.69099808 | Precision: 0.68819459 | Recall: 0.69844648 | F1: 0.69328264
Test  | Acc: 0.69631837 | Precision: 0.69474351 | Recall: 0.70036178 | F1: 0.69754133
current model saved to /home/paperspace/Documents/BioSentVec/svm_models/c_0.05_g_0.01_svm.sav
************************************************************************
c = 0.05
gamma = 0.1
Train  | Acc: 0.67640593 | Precision: 0.63345673 | Recall: 0.83731650 | F1: 0.72125840
Validation  | Acc: 0.65535220 | Precision: 0.61594663 | Recall: 0.82528197 | F1: 0.70541155
Test  | Acc: 0.65801234 | Precision: 0.61870504 | Recall: 0.82357948 | F1: 0.70659120
current model saved to /home/paperspace/Documents/BioSentVec/svm_models/c_0.05_g_0.1_svm.sav
************************************************************************
c = 0.05
gamma = 1
Train  | Acc: 0.50070917 | Precision: 0.50035484 | Recall: 1.00000000 | F1: 0.66698200
Validation

Validation  | Acc: 0.50010641 | Precision: 0.50005321 | Recall: 1.00000000 | F1: 0.66671396
Test  | Acc: 0.50031922 | Precision: 0.50015966 | Recall: 1.00000000 | F1: 0.66680857
current model saved to /home/paperspace/Documents/BioSentVec/svm_models/c_0.2_g_1_svm.sav
************************************************************************
c = 0.2
gamma = 10
Train  | Acc: 0.99996454 | Precision: 0.99992909 | Recall: 1.00000000 | F1: 0.99996454
Validation  | Acc: 0.68110236 | Precision: 0.68852459 | Recall: 0.66141732 | F1: 0.67469880
Test  | Acc: 0.67024899 | Precision: 0.67714792 | Recall: 0.65077676 | F1: 0.66370049
current model saved to /home/paperspace/Documents/BioSentVec/svm_models/c_0.2_g_10_svm.sav
************************************************************************
c = 0.2
gamma = 100
Train  | Acc: 0.99996454 | Precision: 0.99992909 | Recall: 1.00000000 | F1: 0.99996454
Validation  | Acc: 0.51053416 | Precision: 0.50534846 | Recall: 0.99531815 | F1: 0.67034542
Test  | Acc:

In [16]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

X = train_arrays
y = train_labels

for k in list(range(5, 100, 5)):
    print(k)
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X, y) 
    predition = neigh.predict(test_arrays)
    print(neigh.score(test_arrays, test_labels))
    p = precision_score(test_labels, predition)
    r = recall_score(test_labels, predition)
    print(p)
    print(r)
    f = 2 * p * r / (p + r)
    print(f)
    print(neigh.score(train_arrays, train_labels))

5
0.7334869431643625
0.7512396694214876
0.6981566820276498
0.7237261146496815
0.8118385114373506
10
0.7131336405529954
0.7610536218250236
0.6213517665130568
0.6841437632135308
0.7690764766131786
15
0.7265745007680492
0.7478991596638656
0.6835637480798771
0.7142857142857142
0.7682229429839535
20
0.7242703533026114
0.7669104204753199
0.6443932411674347
0.7003338898163606
0.7549931717309663
25
0.7296466973886329
0.7559931506849316
0.6781874039938556
0.7149797570850203
0.7567429156708775
30
0.7238863287250384
0.7614349775784753
0.652073732718894
0.7025237898220935
0.7476954592010925
35
0.7208141321044547
0.7497827975673328
0.662826420890937
0.7036282103546677
0.7487623762376238
40
0.7188940092165899
0.7572202166064982
0.6443932411674347
0.6962655601659752
0.7416780471150564
45
0.7185099846390169
0.748471615720524
0.6582181259600615
0.7004495300367797
0.7449641515875726
50
0.7162058371735791
0.757548032936871
0.6359447004608295
0.6914405010438414
0.7399283031751451
55
0.7185099846390169
0.7

In [22]:
# ensemble learning
from sklearn import svm
abs_dir = "/home/paperspace/Documents/BioSentVec/"
lr_dir = abs_dir + "lg_models/"
rf_dir = abs_dir + "rf_models/"
svm_dir = abs_dir + "svm_models_linear/"# svm_models_linear/c_0.1_svm.sav

lr_model_1 = pickle.load(open(lr_dir + "0.01_lr.sav", 'rb'))
lr_model_2 = pickle.load(open(lr_dir + "0.1_lr.sav", 'rb'))
lr_model_3 = pickle.load(open(lr_dir + "1_lr.sav", 'rb'))
lr_model_4 = pickle.load(open(lr_dir + "10_lr.sav", 'rb'))
lr_model_5 = pickle.load(open(lr_dir + "100_lr.sav", 'rb'))
lr_model_6 = pickle.load(open(lr_dir + "1000_lr.sav", 'rb'))
#rf_model = pickle.load(open(rf_dir + "d_6_200_rf.sav", 'rb'))
svm_model_1 = pickle.load(open(svm_dir + "c_0.1_svm.sav", 'rb'))
svm_model_2 = pickle.load(open(svm_dir + "c_0.01_svm.sav", 'rb'))
#svm = pickle.load(open(, 'rb'))

In [24]:
Y_train_pred_lr = lr_model_1.predict(train_arrays)
print(type(Y_train_pred_lr[0]))
Y_train_pred_s1 = svm_model_1.predict(train_arrays)
print(type(Y_train_pred_s1))
Y_train_pred_s2 = svm_model_2.predict(train_arrays)
print(type(Y_train_pred_s2))
Y_train_pred = np.add(Y_train_pred_lr, Y_train_pred_s1,Y_train_pred_s2)
print(Y_train_pred)
Y_train_pred[Y_train_pred >= 2.0] = 1.0
print(Y_train_pred)
acc_train, pre_train, rec_train, f1_train = accuracy_score(train_labels, Y_train_pred), precision_score(train_labels, Y_train_pred), recall_score(train_labels, Y_train_pred), f1_score(train_labels, Y_train_pred)
print('Train  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_train, pre_train, rec_train, f1_train))

<class 'numpy.float64'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[2. 0. 2. ... 0. 2. 0.]
[1. 0. 1. ... 0. 1. 0.]
Train  | Acc: 0.71771506 | Precision: 0.70420380 | Recall: 0.75079782 | F1: 0.72675476


In [69]:
Y_val_pred_lr = (lr_model_1.predict_proba(val_arrays) >= 0.3).astype(int)
Y_val_pred_lr = Y_val_pred_lr[:, 1]
print(Y_val_pred_lr)
Y_val_pred_s1 = svm_model_1.predict(val_arrays)
print(Y_val_pred_s1)
Y_val_pred_s2 = svm_model_2.predict(val_arrays)
print(type(Y_val_pred_s2))
Y_val_pred = np.add(Y_val_pred_lr, Y_val_pred_s1,Y_val_pred_s2)
print(Y_val_pred)
Y_val_pred[Y_val_pred >= 2.0] = 1.0
print(Y_val_pred)
acc_val, pre_val, rec_val, f1_val = accuracy_score(val_labels, Y_val_pred), precision_score(val_labels, Y_val_pred), recall_score(val_labels, Y_val_pred), f1_score(val_labels, Y_val_pred)
print('val  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_val, pre_val, rec_val, f1_val))

[1 0 0 ... 1 0 1]
[1. 0. 0. ... 0. 0. 1.]
<class 'numpy.ndarray'>
[2. 0. 0. ... 1. 0. 2.]
[1. 0. 0. ... 1. 0. 1.]
val  | Acc: 0.64183869 | Precision: 0.59471366 | Recall: 0.89061502 | F1: 0.71319018


In [17]:
from sklearn.metrics import classification_report

In [25]:
Y_train_pred_lr_1 = (lr_model_1.predict_proba(train_arrays) >= 0.43).astype(int)
Y_train_pred_lr_1 = Y_train_pred_lr_1[:, 1]
Y_train_pred_lr_2 = (lr_model_2.predict_proba(train_arrays) >= 0.57).astype(int)
Y_train_pred_lr_2 = Y_train_pred_lr_2[:, 1]
#Y_train_pred_lr_3 = (lr_model_3.predict_proba(train_arrays) >= 0.47).astype(int)
#Y_train_pred_lr_3 = Y_train_pred_lr_3[:, 1]
Y_train_pred_s1 = svm_model_1.predict(train_arrays)
#Y_train_pred_s2 = svm_model_2.predict(train_arrays)
Y_train_pred = np.add(Y_train_pred_lr_1,Y_train_pred_lr_2)
#Y_train_pred = np.add(Y_train_pred, Y_train_pred_lr_3)
Y_train_pred = np.add(Y_train_pred, Y_train_pred_s1)
#Y_train_pred = np.add(Y_train_pred, Y_train_pred_s2)
Y_train_pred[Y_train_pred >= 2.0] = 1.0
Y_train_pred=Y_train_pred.astype(int) 
train_labels=train_labels.astype(int) 
print(classification_report(train_labels,Y_train_pred,labels=[0,1]))
print(accuracy_score(train_labels, Y_train_pred))

              precision    recall  f1-score   support

           0       0.76      0.59      0.66     14101
           1       0.67      0.81      0.73     14101

   micro avg       0.70      0.70      0.70     28202
   macro avg       0.71      0.70      0.70     28202
weighted avg       0.71      0.70      0.70     28202

0.701439614211758


In [26]:
Y_val_pred_lr_1 = (lr_model_1.predict_proba(val_arrays) >= 0.43).astype(int)
Y_val_pred_lr_1 = Y_val_pred_lr_1[:, 1]
Y_val_pred_lr_2 = (lr_model_2.predict_proba(val_arrays) >= 0.57).astype(int)
Y_val_pred_lr_2 = Y_val_pred_lr_2[:, 1]
#Y_val_pred_lr_3 = (lr_model_3.predict_proba(val_arrays) >= 0.47).astype(int)
#Y_val_pred_lr_3 = Y_val_pred_lr_3[:, 1]
Y_val_pred_s1 = svm_model_1.predict(val_arrays)
#Y_val_pred_s2 = svm_model_2.predict(val_arrays)
Y_val_pred = np.add(Y_val_pred_lr_1,Y_val_pred_lr_2)
#Y_val_pred = np.add(Y_val_pred, Y_val_pred_lr_3)
Y_val_pred = np.add(Y_val_pred, Y_val_pred_s1)
#Y_val_pred = np.add(Y_val_pred, Y_val_pred_s2)
Y_val_pred[Y_val_pred >= 2.0] = 1.0
Y_val_pred=Y_val_pred.astype(int) 
val_labels=val_labels.astype(int) 
print(classification_report(val_labels,Y_val_pred,labels=[0,1]))
print(accuracy_score(val_labels, Y_val_pred))

              precision    recall  f1-score   support

           0       0.74      0.57      0.64      4699
           1       0.65      0.80      0.72      4699

   micro avg       0.69      0.69      0.69      9398
   macro avg       0.70      0.69      0.68      9398
weighted avg       0.70      0.69      0.68      9398

0.6855713981698234


In [ ]:
# RCNN
rcnn_model_dir = '/home/paperspace/Documents/RCNN-421-BioSentVec/runs-ns/1571088121'


In [36]:
Y_test_pred_lr_1 = (lr_model_1.predict_proba(test_arrays) >= 0.42).astype(int)
Y_test_pred_lr_1 = Y_test_pred_lr_1[:, 1]
Y_test_pred = Y_test_pred_lr_1
acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred), recall_score(test_labels, Y_test_pred), f1_score(test_labels, Y_test_pred)
print(acc_test)
print(pre_test)
print(rec_test)
Y_test_pred_lr_2 = (lr_model_2.predict_proba(test_arrays) >= 0.57).astype(int)
Y_test_pred_lr_2 = Y_test_pred_lr_2[:, 1]
#Y_test_pred_lr_3 = (lr_model_3.predict_proba(test_arrays) >= 0.47).astype(int)
#Y_test_pred_lr_3 = Y_test_pred_lr_3[:, 1]
Y_test_pred_s1 = svm_model_1.predict(test_arrays)
#Y_test_pred_s2 = svm_model_2.predict(test_arrays)
Y_test_pred = np.add(Y_test_pred_lr_1,Y_test_pred_lr_2)
#Y_test_pred = np.add(Y_test_pred, Y_test_pred_lr_3)
Y_test_pred = np.add(Y_test_pred, Y_test_pred_s1)
#Y_test_pred = np.add(Y_test_pred, Y_test_pred_s2)
Y_test_pred[Y_test_pred >= 2.0] = 1.0
Y_test_pred=Y_test_pred.astype(int) 
test_labels=test_labels.astype(int) 
acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred), recall_score(test_labels, Y_test_pred), f1_score(test_labels, Y_test_pred)
print(acc_test)
print(pre_test)
print(rec_test)
#print(classification_report(test_labels,Y_test_pred,labels=[0,1]))

0.6895084060438391
0.6541991341991342
0.8040008512449457
0.6894020004256225
0.653342522398346
0.8069802085550117


In [107]:
Y_test_pred_lr_1 = (lr_model_1.predict_proba(test_arrays) >= 0.44).astype(int)
Y_test_pred_lr_1 = Y_test_pred_lr_1[:, 1]
Y_test_pred_lr_2 = (lr_model_2.predict_proba(test_arrays) >= 0.44).astype(int)
Y_test_pred_lr_2 = Y_test_pred_lr_2[:, 1]
Y_test_pred_lr_3 = (lr_model_3.predict_proba(test_arrays) >= 0.44).astype(int)
Y_test_pred_lr_3 = Y_test_pred_lr_3[:, 1]
'''
Y_test_pred_lr_4 = (lr_model_4.predict_proba(test_arrays) >= 0.44).astype(int)
Y_test_pred_lr_4 = Y_test_pred_lr_4[:, 1]
Y_test_pred_lr_5 = (lr_model_5.predict_proba(test_arrays) >= 0.44).astype(int)
Y_test_pred_lr_5 = Y_test_pred_lr_5[:, 1]
Y_test_pred_lr_6 = (lr_model_6.predict_proba(test_arrays) >= 0.44).astype(int)
Y_test_pred_lr_6 = Y_test_pred_lr_6[:, 1]
'''
Y_test_pred_s1 = svm_model_1.predict(test_arrays)
Y_test_pred_s2 = svm_model_2.predict(test_arrays)

Y_test_pred = np.add(Y_test_pred_lr_1,Y_test_pred_lr_2)
Y_test_pred = np.add(Y_test_pred, Y_test_pred_lr_3)
#Y_test_pred = np.add(Y_test_pred, Y_test_pred_lr_4)
#Y_test_pred = np.add(Y_test_pred, Y_test_pred_lr_5)
#Y_test_pred = np.add(Y_test_pred, Y_test_pred_lr_6)
Y_test_pred = np.add(Y_test_pred, Y_test_pred_s1)
Y_test_pred = np.add(Y_test_pred, Y_test_pred_s2)
print(Y_test_pred)
Y_test_pred[Y_test_pred > 3.0] = 1
Y_test_pred=Y_test_pred.astype(int) 
test_labels=test_labels.astype(int) 
acc_test, pre_test, rec_test, f1_test = accuracy_score(test_labels, Y_test_pred), precision_score(test_labels, Y_test_pred,average='micro'), recall_score(test_labels, Y_test_pred,average='micro'), f1_score(test_labels, Y_test_pred,average='micro')
print('test  | Acc: %.8f | Precision: %.8f | Recall: %.8f | F1: %.8f' % (acc_test, pre_test, rec_test, f1_test))

[5. 0. 3. ... 0. 0. 5.]
test  | Acc: 0.65939562 | Precision: 0.65939562 | Recall: 0.65939562 | F1: 0.65939562
